In [1]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

In [2]:
import numpy as np
import pandas as pd
#import tensorflow as tf
import matplotlib.pyplot as plt
import json
import re
import pickle

In [3]:
import collections
import math
import os
import random
import zipfile

In [4]:
data_in = '/backup/namuGrim/data/'

# ko-Wikipedia

In [5]:
kowiki = pickle.load(open(data_in + 'kowiki.p','rb'))

In [6]:
print (kowiki.shape)
kowiki.head(3)

(964236, 2)


,title,text
0,위키백과:대문,"<!-- 머릿글 -->\n<div id=""mp_header"" class=""mp_ou..."
1,지미 카터,{{국가원수 정보\n| 이름 = 제임스 얼 카터 주니어\n| 원래 이름 = {{la...
2,수학,"'''수학'''(數學)은 [[양]], [[구조]], [[공간]], [[변화]] 등 ..."


# Namu Wiki

In [7]:
namuwiki = pickle.load(open(data_in + 'namuwiki.p', 'rb'))

In [8]:
print (namuwiki.shape)
namuwiki.head(3)

(842910, 2)


,text,title
0,#redirect 느낌표\n,!
1,[[파일:3444050440.jpg]]\n([[신 세계수의 미궁 2]]에서 뜬 !!...,!!아앗!!
2,"[include(틀:다른 뜻1, other1=말줄임표 등으로 사용하는 용어, rd1...",“……”


In [9]:
def tokenize(content):
    return ["{}/{}".format(word, tag) for word, tag in tagger.pos(content) if tag == 'Noun']

In [10]:
class SentenceReader:
    def __init__ (self, frame, filt=lambda x: tokenize(x.text)):
        self.frame = frame
        self.filt = filt
    
    def __iter__ (self):
        for idx, content in self.frame.iterrows():
            yield self.filt(content)        

In [11]:
from konlpy.tag import Twitter
tagger = Twitter()

In [12]:
import gensim

In [13]:
model = gensim.models.Word2Vec()

In [14]:
sentence_vocab = SentenceReader(kowiki)
sentence_train = SentenceReader(kowiki)

In [15]:
model.build_vocab(sentence_vocab)

RuntimeError: No matching overloads found. at native/common/jp_method.cpp:117

In [ ]:
model.train(sentence_train)

In [ ]:
model.vocab

In [ ]:
pat_redirect = re.compile('^#redirect (.+)')
def check_redirect(text):
    match = pat_redirect.match(text)
    if match:
        return match.group(1)
    else:
        return False

In [ ]:
def get_article(title):
    p = frame.loc[frame['title'] == title]
    return p.text.values[0]    

In [ ]:
pat_top = re.compile(r'\* 상위 문서 \: (.+)')
pat_bracket = re.compile(r'\[\[(.+?)\]\]')

def get_top(text):
    matches = pat_top.finditer(text)
    ret = []
    for match in matches:
        bracket = pat_bracket.search(match.group(1))
        if bracket:
            ret.append(bracket.group(1))
        else:
            ret.append(match.group(1))
    return ret

In [ ]:
cate = ['배우/한국', '가수/한국']

In [ ]:
actors = []
for idx, context in frame.iterrows():
    if any(actor in get_top(context.text) for actor in cate):
        actors.append(context.title)

In [ ]:
print (len(actors))
for actor in actors[:5]:
    print (actor)

##  save actors

In [ ]:
with open('../data/actors.txt', 'w') as f:
    for actor in actors:
        f.write(actor + r'\n')

In [ ]:
pat_bracket = re.compile(r'\[\[(.+?)\]\]')
pat_file = re.compile(r'\[\[파일:(.+)\]\]')
pat_link = re.compile(r'\[\[(.+?)\|(.+?)\]\]')
pat_comment = re.compile(r'\[\*(.+?)\]')
pat_high = re.compile(r'\{\{\{(.+?)\}\}\}')

def bracket_filter(text):
    ret = ""
    match = pat_file.match(text)
    if match: 
        ret = ""
    else:
        match = pat_link.match(text)
        if match: 
            ret = match.group(2)
        else:
            ret = text[2:-2]
    return ret

def context_filter(text):
    # find bracket
    delc = 0
    matches = pat_bracket.finditer(text)
    for match in matches:
        conv = bracket_filter(match.group(0))
        text = text[:match.start() - delc] + conv +  text[match.end() - delc:]
        delc += len(match.group(0)) - len(conv)
        
    # comments
    delc = 0
    matches = pat_comment.finditer(text)
    for match in matches:
        text = text[:match.start() - delc] + match.group(0) +  text[match.end() - delc:]
        delc += 3
        
    # find highlight
    delc = 0
    matches = pat_high.finditer(text)
    for match in matches:
        text = text[:match.start() - delc] + match.group(1) +  text[match.end() - delc:]
        delc += 6
    
    return text

In [ ]:
def tokenize(doc):
    return ['/'.join(t) for t in Twitter.pos(doc)]

In [ ]:
def rawnize(title):
    return get_article(title)

In [ ]:
def textnize(title):
    text = get_article(title)
    return context_filter(text)

In [ ]:
for idx, db in frame.iterrows():
    text = db.text
    chk = check_redirect(text)
    if chk:
        text = get_article(chk)
    context = context_filter(text)
    print (context)
    break